# **Introduction to Verlet integration (exmaple: 2-body system)**

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

<p style="text-align: justify;font-size:15px"> 
In the molecular dyanmics simulations, the velocity, position and acceleration are sloved numerically by using the Verlet algorithm.
<p>

<p style="text-align: justify;font-size:20px"> 
$\mathbf{r}(t+\Delta t) = 2\mathbf{r}(t)-\mathbf{r}(t-\Delta t) + \mathbf{a}(t)\Delta t^2 + O(\Delta t^4)$
<p>

<p style="text-align: justify;font-size:20px">
$\mathbf{a}(t) = - \frac{1}{m}\nabla\mathbf{v}(\mathbf{r}(t))$
<p>
    
<p style="text-align: justify;font-size:23px">
$\mathbf{v}(t+ \Delta t) = \frac{\mathbf{r}(t+\Delta t)-\mathbf{r}(t-\Delta t)}{2\Delta t}$
<p>
    
<p style="text-align: justify;font-size:15px"> 
Since this is a numerical method, the choosing of the time step $\Delta t$ is very important. 
    If one chooses a very small time step to get accurate results, the simulations will take 
    long time to run. On the other hand, if one chooses a very large time step, the molecular 
    simulations will go to wrong results.
<p>
    
<p style="text-align: justify;font-size:15px"> 
Here, we domonstrates a two-body system (sun and earth) with the Verlet algorithm. The sun is 
    fixed at the centre positon. By tunning the time step with the slider, one can see how the
    choosing of the time step $\Delta t$ influences the trajectory of the earth.
<p>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from ipywidgets import Button, IntSlider
from numpy.linalg import norm

In [ ]:
def compute_force(r1, r2, m1, m2):
    x = G*m1*m2/np.linalg.norm(r1-r2)**2
    v = (r2-r1)/np.linalg.norm(r2-r1)
    return x*v

In [ ]:
def compute_acceleration(f, m):
    return f/m

In [ ]:
def verlet_update(r1, r2, v, m1, m2, dt, a):
    
    rn = 2.0*r1[-1] - r1[-2]  + dt*dt*a
    vn = (rn - r1[-2])/(2*dt)
    
    r1[-2] = r1[-1]
    r1[-1] = rn
    
    f = compute_force(rn, r2, m1, m2)
    a = compute_acceleration(f, m1)
    
    return r1, vn, a
    
    #return np.append(r1, [rn], axis=0), vn
  

In [ ]:
%matplotlib widget

m1 = 1.0
m2 = 1000.0
G = 0.2
v = [0.0, 3.0]
dt = 0.01

r1 = np.array([[10, 0],[10.0, 3.0*dt]])
r2 = np.array([0, 0])

f = compute_force(r1[-1], r2, m1, m2)
a = compute_acceleration(f, m1)

fig, ax = plt.subplots()
fig.canvas.header_visible = False

sun = plt.scatter([0.0],[0.0], s=23**2, 
               marker="o", linewidth=0, color='gold', label = 'Sun')
earth, = plt.plot([10.0],[0.0], 'r.', alpha = 1.0, markersize=15, label = 'Earth')
origin = plt.plot([10.0],[0.0], 'kx', alpha = 1.0, markersize=15, label = 'Origin point')
avector = plt.quiver([10.0], [0], [0], [0], color = ['blue'], units='inches', scale = 10.0, label="a")
fvector = plt.quiver([10.0], [0], [0], [0], color = ['gold'], units='inches', scale = 10.0, label="v")
trace, = plt.plot([10.0], [0.0], "k", linewidth=0.5)
title = ax.set_title("")

plt.legend(loc=2, fontsize=12)

plt.xlim(-11.0, 11.0)
plt.ylim(-11.0, 11.0)
plt.grid()


xt = [];
yt = [];

def init():
    return (earth, avector, fvector, trace)

    
def update(i):
    global r1, v, a, xt, yt
    r1, v, a = verlet_update(r1, r2, v, m1, m2, dt, a)

    earth.set_data([r1[-1, 0]], [r1[-1, 1]])
    xt.append(r1[-1, 0])
    yt.append(r1[-1, 1])
    trace.set_data(xt, yt)
    avector.set_offsets(np.array([r1[-1, 0], r1[-1, 1]]))
    fvector.set_offsets(np.array([r1[-1, 0], r1[-1, 1]]))
    avector.set_UVC(a[0], a[1])
    fvector.set_UVC(v[0], v[1])
    
    title.set_text("v(t) = %-5.3f" % norm(v))

    return (earth, avector, fvector, trace)


anim = FuncAnimation(fig, update, frames = 20000, interval = 10,
                    init_func=init, blit=True, repeat = True)


def onClick(event):
    if button_pause.description == "Pause":
        button_pause.description = "Play"
        anim.event_source.stop()
    else:
        button_pause.description = "Pause"
        anim.event_source.start()
        

def clear_data(b):
    global xt, yt
    
    xt = [];
    yt = [];

button_pause = Button(description="Pause");
button_pause.on_click(onClick)
button_clear = Button(description="Clear trace")
button_clear.on_click(clear_data)

plt.show()

def on_dt_change(c):
    global dt, r1, xt, yt
    
    anim.event_source.stop()
    
    dt = w_dt.value*0.01
    r1 = np.array([[10, 0],[10.0, 3.0*dt]])
    
    xt = []
    yt = []
    
    anim.event_source.start()

w_dt = IntSlider(value = 1, min = 1, max = 20, step=1, description="dt: ")
w_dt.observe(on_dt_change, names='value')

display(w_dt, button_pause, button_clear)